###General Instructions
In this assignment, you will need to complete the code samples where indicated to accomplish the given objectives. **Be sure to run all cells** and export this notebook as an HTML with results included.  Upload the exported HTML file to Canvas by the assignment deadline.

####Assignment
Unlike previous exercises, you will not be provided any sample code from which to work.  You are given a starting point and an expected outcome.  Use what you have learned in class and in labs to complete the exercise as outlined below. Break your logic up into cells that tackle webpage retrieval, HTML parsing, and DataFrame assembly and persistence.

Board Game Geek is an online site that provides rankings for various boardgames.  A complete listing of ranked games is provided here: https://boardgamegeek.com/browse/boardgame

The data on this page is organized in a simple table format. The Title field is divided into the game's title and the year it was published. The title itself provides a link to more information on the game.

Write a routine to scrape this data from the page and persist it as a partitioned Parquet table named games.board_games.  Use the current date as the partitioning field.

The fields required in the final table are:

* Rank - the integer value from the Board Game Rank column
* ImageURL - the URL (as provided) of the image presented in the second column 
* Title - the title of the game
* YearPublished - the year the game was published
* GeekRating - the float value presented in the Geek Rating field
* AvgRating - the float value presented in the Avg Rating field
* NumVoters - the integer value presented in the NumVoters field

(You can ignore the Shop field)

While the Board Game Geek site has multiple pages of rankings, limit your scraping to the games presented on the page referenced above.  (In other words, don't hit the other 1000+ pages of rankings.)

Once you've populated your table, run the two SQL statements presented at the bottom of this notebook.  Save your notebook (with the results) as an HTML file and load it to Canvas.

In [0]:
# run this code to make sure beautiful soup is installed
dbutils.library.installPyPI('beautifulsoup4')
dbutils.library.restartPython()

dbutils.library APIs are deprecated and will be removed in a future DBR release. You can use %pip commands to install notebook scoped python libraries.
PyPI package beautifulsoup4 has been installed already. The previously installed package is `beautifulsoup4`. To resolve this issue, detach and re-attach the notebook to create a new environment or rename the package.


In [0]:
%sql -- create games database
create database if not exists games;

In [0]:
from urllib import request as url

# retrieve the web page
request = url.Request('https://boardgamegeek.com/browse/boardgame')

# submit this request and retrieve a result
response = url.urlopen(request)

html = response.read()

print(html)

b'<!DOCTYPE html>\n<html ng-app="GeekApp" lang="en-US" ng-cloak>\n<head>\n\t<meta charset=\'utf-8\'>\n\t<meta id="vp" name="viewport" content="width=device-width, initial-scale=1.0">\n\t\t\t<script>\n\t\t\twindow.addEventListener( \'DOMContentLoaded\',  function() {\n\t\t\t\tvar width = document.documentElement.clientWidth || window.innerWidth;\n\t\t\t\tif (width < 960) {\n\t\t\t\t\tvar mvp = document.getElementById(\'vp\');\n\t\t\t\t\t// android debugging\n\t\t\t\t\tmvp.setAttribute(\'content\',\'width=960\');\n\t\t\t\t}\n\t\t\t});\n\t\t</script>\n\t\t<meta content=\'yes\' name=\'apple-mobile-web-app-capable\'>\n\t<meta content=\'IE=edge,chrome=1\' http-equiv=\'X-UA-Compatible\'>\n\n\t\t\t<title>Browse Board Games | BoardGameGeek</title>\n\t\n\t\n<link rel="apple-touch-icon" \thref="https://cf.geekdo-static.com/icons/touch-icon180.png" />\n<link rel="shortcut icon" \t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="icon" \t\t\t\t\thref="https://

In [0]:
from bs4 import BeautifulSoup

# load raw html into beautiful soup
soup = BeautifulSoup(html)

# print beautiful soup html
print( soup )

<!DOCTYPE html>

<html lang="en-US" ng-app="GeekApp" ng-cloak="">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" id="vp" name="viewport"/>
<script>
			window.addEventListener( 'DOMContentLoaded',  function() {
				var width = document.documentElement.clientWidth || window.innerWidth;
				if (width < 960) {
					var mvp = document.getElementById('vp');
					// android debugging
					mvp.setAttribute('content','width=960');
				}
			});
		</script>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>Browse Board Games | BoardGameGeek</title>
<link href="https://cf.geekdo-static.com/icons/touch-icon180.png" rel="apple-touch-icon"/>
<link href="https://cf.geekdo-static.com/icons/favicon2.ico" rel="shortcut icon" type="image/ico"/>
<link href="https://cf.geekdo-static.com/icons/favicon2.ico" rel="icon" type="image/ico"/>
<link href="/game-opensearch.xml" rel="search" title="BGG

In [0]:
# extract the data from the web page     
tables = soup.find_all('table')
table = tables[0]

print(table) #verify table was pulled


<table cellpadding="0" cellspacing="1" class="collection_table" id="collectionitems" width="100%">
<tr>
<th class="collection_bggrating">
<a href="/browse/boardgame?sort=rank&amp;sortdir=desc">Board Game Rank<img alt="ascending sort" border="0" hspace="5" src="https://cf.geekdo-static.com/images/collection/arrow_up.gif"/></a>
</th>
<th class="collection_thumbnail">
<span class="sr-only">Thumbnail image</span>
</th>
<th class="collection_title">
<a href="/browse/boardgame?sort=title">Title</a>
</th>
<th class="collection_bggrating">
<a href="/browse/boardgame?sort=bggrating">Geek Rating</a>
</th>
<th class="collection_bggrating">
<a href="/browse/boardgame?sort=avgrating">Avg Rating</a>
</th>
<th class="collection_bggrating">
<a href="/browse/boardgame?sort=numvoters">Num Voters</a>
</th>
<th class="collection_shop">
			Shop
		</th>
</tr>
<tr id="row_">
<td align="center" class="collection_rank">
<a name="1"></a>			1			
					</td>
<td class="collection_thumbnail">
<a href="/boardgame/17

In [0]:
#pull all headers for the table
header = []
for th in table.tr.find_all('th'):
  header += [th.text]
  
header #verify headers were pulled

Out[4]: ['\nBoard Game Rank\n',
 '\nThumbnail image\n',
 '\nTitle\n',
 '\nGeek Rating\n',
 '\nAvg Rating\n',
 '\nNum Voters\n',
 '\n\t\t\tShop\n\t\t']

In [0]:
all_data = []
 
for table in tables:
  table_data = []
  for tr in table.find_all('tr', attrs={'id': 'row_'}):
    row_data = []
    for td in tr.find_all('td'):
      value = ''
      if td.a and td.a.img:
        value = td.a.img['src']
      else:
        value = td.text
      row_data += [value]
    table_data += [row_data]
  all_data += table_data
  
all_data #verify the table data was pulled

Out[5]: [['\n\t\t\t1\t\t\t\n\t\t\t\t\t',
  'https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZfZmuA__micro/img/sQyh47ClBO3d5sxPm73hMvM-JV4=/fit-in/64x64/filters:strip_icc()/pic2437871.jpg',
  '\n\n\nGloomhaven\n(2017)\n\n\n\t\t\tVanquish monsters with strategic cardplay. Fulfill your quest to leave your legacy!\n\t\t\n',
  '\n\t\t\t8.499\t\t',
  '\n\t\t\t8.73\t\t',
  '\n\t\t\t49467\t\t',
  '\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t[Shop]\n\t\t\t\t\t'],
 ['\n\t\t\t2\t\t\t\n\t\t\t\t\t',
  'https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__micro/img/n6-sXYD6XXZoqIxq4P6AG7VPCuA=/fit-in/64x64/filters:strip_icc()/pic2452831.png',
  '\n\n\nPandemic Legacy: Season 1\n(2015)\n\n\n\t\t\tMutating diseases are spreading around the world - can your team save humanity?\n\t\t\n',
  '\n\t\t\t8.437\t\t',
  '\n\t\t\t8.59\t\t',
  '\n\t\t\t46063\t\t',
  '\n\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t[Shop]\n\t\t\t\t\t'],
 ['\n\t\t\t3\t\t\t\n\t\t\t\t\t',
  'https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__micro/im

In [0]:
# convert the data to a Spark SQL table named board_games in the games database
import pyspark.sql.functions as f
from pyspark.sql.types import *
 
board_games = spark.createDataFrame(all_data, schema=header)
 
for col_name in board_games.columns:
  new_col_name = col_name.title().strip().replace(' ', '')
  board_games = board_games.withColumnRenamed(col_name, new_col_name)
  
board_games = board_games.drop('Shop')

display(board_games) #verify Shop was dropped

BoardGameRank,ThumbnailImage,Title,GeekRating,AvgRating,NumVoters
1,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZfZmuA__micro/img/sQyh47ClBO3d5sxPm73hMvM-JV4=/fit-in/64x64/filters:strip_icc()/pic2437871.jpg,Gloomhaven (2017) Vanquish monsters with strategic cardplay. Fulfill your quest to leave your legacy!,8.499,8.73,49467
2,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__micro/img/n6-sXYD6XXZoqIxq4P6AG7VPCuA=/fit-in/64x64/filters:strip_icc()/pic2452831.png,Pandemic Legacy: Season 1 (2015) Mutating diseases are spreading around the world - can your team save humanity?,8.437,8.59,46063
3,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__micro/img/4Od3GYCiqptga0VbmyumPbJlBsU=/fit-in/64x64/filters:strip_icc()/pic3490053.jpg,"Brass: Birmingham (2018) Build networks, grow industries, and navigate the world of the Industrial Revolution.",8.422,8.66,27313
4,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZQ4rxw__micro/img/LUkXZhd1TO80eCiXMD3-KfnzA6k=/fit-in/64x64/filters:strip_icc()/pic3536616.jpg,Terraforming Mars (2016) Compete with rival CEOs to make Mars habitable and build your corporate empire.,8.269,8.41,77092
5,https://cf.geekdo-images.com/_HhIdavYW-hid20Iq3hhmg__micro/img/OdKjWiFsNvQAfJfXXSITttiozWE=/fit-in/64x64/filters:strip_icc()/pic5055631.jpg,Gloomhaven: Jaws of the Lion (2020) Vanquish monsters with strategic cardplay in a 25-scenario Gloomhaven campaign.,8.267,8.65,18306
6,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE-FgFA__micro/img/2gymaKs35_2yj7eyyA6cYyVmd9c=/fit-in/64x64/filters:strip_icc()/pic3727516.jpg,"Twilight Imperium: Fourth Edition (2017) Build an intergalactic empire through trade, research, conquest and grand politics.",8.256,8.66,16792
7,https://cf.geekdo-images.com/hGWFm3hbMlCDsfCsauOQ4g__micro/img/odWj2bEnZi9dEcQQrOtYj1e0QLE=/fit-in/64x64/filters:strip_icc()/pic5375625.png,"Gaia Project (2017) Expand, research, upgrade, and settle the galaxy with one of 14 alien species.",8.173,8.47,20213
8,https://cf.geekdo-images.com/7SrPNGBKg9IIsP4UQpOi8g__micro/img/nEvTiCkWpT-ymH4bstc9c335TtQ=/fit-in/64x64/filters:strip_icc()/pic4325841.jpg,Star Wars: Rebellion (2016) Strike from your hidden base as the Rebels—or find and destroy it as the Empire.,8.169,8.42,26313
9,https://cf.geekdo-images.com/a13ieMPP2s0KEaKNYmtH5w__micro/img/HQaHfx5cy6wJtZBMZCbVeNbfdy8=/fit-in/64x64/filters:strip_icc()/pic3615739.png,Spirit Island (2017) Island Spirits join forces using elemental powers to defend their home from invaders.,8.142,8.36,34294
10,https://cf.geekdo-images.com/ImPgGag98W6gpV1KV812aA__micro/img/NT-Av_3kdYUcwuti5ocmIQXow3g=/fit-in/64x64/filters:strip_icc()/pic1215633.jpg,War of the Ring: Second Edition (2011) The Fellowship and the Free Peoples clash with Sauron over the fate of Middle-Earth.,8.141,8.52,16056


In [0]:
#create regex to use for parsing
regex_pattern = r'^(.*)\((\d*)\)(.*)$'
 
# Parsing to create the YearPublished from Title data and dropping the description of the game
parse_board_games = (
  board_games
    .withColumn('Title', f.regexp_replace('Title', r'\n|\t', ''))
    .withColumn('YearPublished', f.regexp_extract('Title', regex_pattern, 2))
    .withColumn('Title', f.regexp_extract('Title', regex_pattern, 1))
)

display(parse_board_games)

BoardGameRank,ThumbnailImage,Title,GeekRating,AvgRating,NumVoters,YearPublished
1,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZfZmuA__micro/img/sQyh47ClBO3d5sxPm73hMvM-JV4=/fit-in/64x64/filters:strip_icc()/pic2437871.jpg,Gloomhaven,8.499,8.73,49467,2017
2,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__micro/img/n6-sXYD6XXZoqIxq4P6AG7VPCuA=/fit-in/64x64/filters:strip_icc()/pic2452831.png,Pandemic Legacy: Season 1,8.437,8.59,46063,2015
3,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__micro/img/4Od3GYCiqptga0VbmyumPbJlBsU=/fit-in/64x64/filters:strip_icc()/pic3490053.jpg,Brass: Birmingham,8.422,8.66,27313,2018
4,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZQ4rxw__micro/img/LUkXZhd1TO80eCiXMD3-KfnzA6k=/fit-in/64x64/filters:strip_icc()/pic3536616.jpg,Terraforming Mars,8.269,8.41,77092,2016
5,https://cf.geekdo-images.com/_HhIdavYW-hid20Iq3hhmg__micro/img/OdKjWiFsNvQAfJfXXSITttiozWE=/fit-in/64x64/filters:strip_icc()/pic5055631.jpg,Gloomhaven: Jaws of the Lion,8.267,8.65,18306,2020
6,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE-FgFA__micro/img/2gymaKs35_2yj7eyyA6cYyVmd9c=/fit-in/64x64/filters:strip_icc()/pic3727516.jpg,Twilight Imperium: Fourth Edition,8.256,8.66,16792,2017
7,https://cf.geekdo-images.com/hGWFm3hbMlCDsfCsauOQ4g__micro/img/odWj2bEnZi9dEcQQrOtYj1e0QLE=/fit-in/64x64/filters:strip_icc()/pic5375625.png,Gaia Project,8.173,8.47,20213,2017
8,https://cf.geekdo-images.com/7SrPNGBKg9IIsP4UQpOi8g__micro/img/nEvTiCkWpT-ymH4bstc9c335TtQ=/fit-in/64x64/filters:strip_icc()/pic4325841.jpg,Star Wars: Rebellion,8.169,8.42,26313,2016
9,https://cf.geekdo-images.com/a13ieMPP2s0KEaKNYmtH5w__micro/img/HQaHfx5cy6wJtZBMZCbVeNbfdy8=/fit-in/64x64/filters:strip_icc()/pic3615739.png,Spirit Island,8.142,8.36,34294,2017
10,https://cf.geekdo-images.com/ImPgGag98W6gpV1KV812aA__micro/img/NT-Av_3kdYUcwuti5ocmIQXow3g=/fit-in/64x64/filters:strip_icc()/pic1215633.jpg,War of the Ring: Second Edition,8.141,8.52,16056,2011


In [0]:
#Cleaning the parsed data

clean_board_games = (
  parse_board_games
    .withColumnRenamed('BoardGameRank', 'Rank')
    .withColumnRenamed('ThumbnailImage', 'ImageURL')
    .withColumn('Rank', f.col('Rank').cast(IntegerType()))
    .withColumn('GeekRating', f.col('GeekRating').cast(FloatType()))
    .withColumn('AvgRating', f.col('AvgRating').cast(FloatType()))
    .withColumn('NumVoters', f.col('NumVoters').cast(IntegerType()))
    .withColumn('YearPublished', f.col('YearPublished').cast(IntegerType()))
)

display(clean_board_games) #verify table is cleaned

Rank,ImageURL,Title,GeekRating,AvgRating,NumVoters,YearPublished
1,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZfZmuA__micro/img/sQyh47ClBO3d5sxPm73hMvM-JV4=/fit-in/64x64/filters:strip_icc()/pic2437871.jpg,Gloomhaven,8.499,8.73,49467,2017
2,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__micro/img/n6-sXYD6XXZoqIxq4P6AG7VPCuA=/fit-in/64x64/filters:strip_icc()/pic2452831.png,Pandemic Legacy: Season 1,8.437,8.59,46063,2015
3,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__micro/img/4Od3GYCiqptga0VbmyumPbJlBsU=/fit-in/64x64/filters:strip_icc()/pic3490053.jpg,Brass: Birmingham,8.422,8.66,27313,2018
4,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZQ4rxw__micro/img/LUkXZhd1TO80eCiXMD3-KfnzA6k=/fit-in/64x64/filters:strip_icc()/pic3536616.jpg,Terraforming Mars,8.269,8.41,77092,2016
5,https://cf.geekdo-images.com/_HhIdavYW-hid20Iq3hhmg__micro/img/OdKjWiFsNvQAfJfXXSITttiozWE=/fit-in/64x64/filters:strip_icc()/pic5055631.jpg,Gloomhaven: Jaws of the Lion,8.267,8.65,18306,2020
6,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE-FgFA__micro/img/2gymaKs35_2yj7eyyA6cYyVmd9c=/fit-in/64x64/filters:strip_icc()/pic3727516.jpg,Twilight Imperium: Fourth Edition,8.256,8.66,16792,2017
7,https://cf.geekdo-images.com/hGWFm3hbMlCDsfCsauOQ4g__micro/img/odWj2bEnZi9dEcQQrOtYj1e0QLE=/fit-in/64x64/filters:strip_icc()/pic5375625.png,Gaia Project,8.173,8.47,20213,2017
8,https://cf.geekdo-images.com/7SrPNGBKg9IIsP4UQpOi8g__micro/img/nEvTiCkWpT-ymH4bstc9c335TtQ=/fit-in/64x64/filters:strip_icc()/pic4325841.jpg,Star Wars: Rebellion,8.169,8.42,26313,2016
9,https://cf.geekdo-images.com/a13ieMPP2s0KEaKNYmtH5w__micro/img/HQaHfx5cy6wJtZBMZCbVeNbfdy8=/fit-in/64x64/filters:strip_icc()/pic3615739.png,Spirit Island,8.142,8.36,34294,2017
10,https://cf.geekdo-images.com/ImPgGag98W6gpV1KV812aA__micro/img/NT-Av_3kdYUcwuti5ocmIQXow3g=/fit-in/64x64/filters:strip_icc()/pic1215633.jpg,War of the Ring: Second Edition,8.141,8.52,16056,2011


In [0]:
spark.conf.set('spark.sql.legacy.parquet.datetimeRebaseModeInWrite', 'CORRECTED')

spark.sql('DROP TABLE IF EXISTS games.board_games')

#Saving dataframe
(
  clean_board_games
    .select(
      'Rank',
      'ImageURL',
      'Title',
      'YearPublished',
      'GeekRating',
      'AvgRating',
      'NumVoters'
      )
    .write
    .format('delta')
    .mode('overwrite')
    .option('overwriteSchema', 'true')
    .saveAsTable('games.board_games')
)

Execute the following SQL statements to validate your results:

In [0]:
%sql -- verify row count

SELECT COUNT(*) FROM games.board_games;

count(1)
100


In [0]:
%sql -- verify values

SELECT * FROM games.board_games ORDER BY rank ASC;

Rank,ImageURL,Title,YearPublished,GeekRating,AvgRating,NumVoters
1,https://cf.geekdo-images.com/sZYp_3BTDGjh2unaZfZmuA__micro/img/sQyh47ClBO3d5sxPm73hMvM-JV4=/fit-in/64x64/filters:strip_icc()/pic2437871.jpg,Gloomhaven,2017,8.499,8.73,49467
2,https://cf.geekdo-images.com/-Qer2BBPG7qGGDu6KcVDIw__micro/img/n6-sXYD6XXZoqIxq4P6AG7VPCuA=/fit-in/64x64/filters:strip_icc()/pic2452831.png,Pandemic Legacy: Season 1,2015,8.437,8.59,46063
3,https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__micro/img/4Od3GYCiqptga0VbmyumPbJlBsU=/fit-in/64x64/filters:strip_icc()/pic3490053.jpg,Brass: Birmingham,2018,8.422,8.66,27313
4,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZQ4rxw__micro/img/LUkXZhd1TO80eCiXMD3-KfnzA6k=/fit-in/64x64/filters:strip_icc()/pic3536616.jpg,Terraforming Mars,2016,8.269,8.41,77092
5,https://cf.geekdo-images.com/_HhIdavYW-hid20Iq3hhmg__micro/img/OdKjWiFsNvQAfJfXXSITttiozWE=/fit-in/64x64/filters:strip_icc()/pic5055631.jpg,Gloomhaven: Jaws of the Lion,2020,8.267,8.65,18306
6,https://cf.geekdo-images.com/_Ppn5lssO5OaildSE-FgFA__micro/img/2gymaKs35_2yj7eyyA6cYyVmd9c=/fit-in/64x64/filters:strip_icc()/pic3727516.jpg,Twilight Imperium: Fourth Edition,2017,8.256,8.66,16792
7,https://cf.geekdo-images.com/hGWFm3hbMlCDsfCsauOQ4g__micro/img/odWj2bEnZi9dEcQQrOtYj1e0QLE=/fit-in/64x64/filters:strip_icc()/pic5375625.png,Gaia Project,2017,8.173,8.47,20213
8,https://cf.geekdo-images.com/7SrPNGBKg9IIsP4UQpOi8g__micro/img/nEvTiCkWpT-ymH4bstc9c335TtQ=/fit-in/64x64/filters:strip_icc()/pic4325841.jpg,Star Wars: Rebellion,2016,8.169,8.42,26313
9,https://cf.geekdo-images.com/a13ieMPP2s0KEaKNYmtH5w__micro/img/HQaHfx5cy6wJtZBMZCbVeNbfdy8=/fit-in/64x64/filters:strip_icc()/pic3615739.png,Spirit Island,2017,8.142,8.36,34294
10,https://cf.geekdo-images.com/ImPgGag98W6gpV1KV812aA__micro/img/NT-Av_3kdYUcwuti5ocmIQXow3g=/fit-in/64x64/filters:strip_icc()/pic1215633.jpg,War of the Ring: Second Edition,2011,8.141,8.52,16056


In [0]:
%sql -- verify partitioning
DESCRIBE EXTENDED games.board_games;

col_name,data_type,comment
Rank,int,
ImageURL,string,
Title,string,
YearPublished,int,
GeekRating,float,
AvgRating,float,
NumVoters,int,
,,
# Partitioning,,
Not partitioned,,
